<a href="https://colab.research.google.com/github/azzindani/01_Data_Analytic/blob/main/French_Bakery_Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 00 Importing Modules & Mounting Drive

In [1]:
!pip install pandas-profiling
!pip install streamlit
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 1.519s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [2]:
import pathlib
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from ydata_profiling import ProfileReport
from tabulate import tabulate
from time import sleep

%matplotlib inline

In [3]:
'''from google.colab import drive
drive.mount('/content/drive') #'''

"from google.colab import drive\ndrive.mount('/content/drive') #"

In [4]:
'''MAIN_PATH = str(pathlib.Path().resolve())
WORK_PATH = MAIN_PATH + '/drive/MyDrive/Workspace'
SOURCE_PATH = WORK_PATH + '/00_Data_Source'
CACHE_PATH = WORK_PATH + '/00_Cache_Data' #'''

"MAIN_PATH = str(pathlib.Path().resolve())\nWORK_PATH = MAIN_PATH + '/drive/MyDrive/Workspace'\nSOURCE_PATH = WORK_PATH + '/00_Data_Source'\nCACHE_PATH = WORK_PATH + '/00_Cache_Data' #"

## 01 Choosing & Importing Dataset

### 01.01 Importing Main Data

In [5]:
dataset_path = 'https://raw.githubusercontent.com/azzindani/00_Data_Source/main/French_Bakery_Sales.csv'
df = pd.read_csv(dataset_path)#, encoding = 'ISO-8859-1')#, sep = '\t')
df.head(2)

,Unnamed: 0,date,time,ticket_number,article,Quantity,unit_price
0,0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €"
1,1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €"


In [6]:
df['unit_price'] = df['unit_price'].str.replace(' €', '')
df['unit_price'] = df['unit_price'].str.replace(',', '.')
df['unit_price'] = df['unit_price'].astype('float')
df.head(2)

,Unnamed: 0,date,time,ticket_number,article,Quantity,unit_price
0,0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,0.9
1,1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,1.2


In [7]:
df['sales'] = df['Quantity'] * df['unit_price']
df.head(2)

,Unnamed: 0,date,time,ticket_number,article,Quantity,unit_price,sales
0,0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,0.9,0.9
1,1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,1.2,3.6


### 01.02 Importing Geo Data

### 01.03 Importing Additional Data (for enrichment)

## 02 Checking Data

In [8]:
df.shape

(234005, 8)

In [9]:
df.columns

Index(['Unnamed: 0', 'date', 'time', 'ticket_number', 'article', 'Quantity',
       'unit_price', 'sales'],
      dtype='object')

In [10]:
df.nunique()

Unnamed: 0       234005
date                600
time                683
ticket_number    136451
article             149
Quantity             40
unit_price          123
sales               357
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     234005 non-null  int64  
 1   date           234005 non-null  object 
 2   time           234005 non-null  object 
 3   ticket_number  234005 non-null  float64
 4   article        234005 non-null  object 
 5   Quantity       234005 non-null  float64
 6   unit_price     234005 non-null  float64
 7   sales          234005 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 14.3+ MB


## 03 Data Structuring

### 03.01 Selecting & Dropping Variables

### 03.02 Cleaning Text Object

#### 03.02.01 Convert Header to Proper Text

In [12]:
for x in df.columns:
  y = x.title()
  df = df.rename(columns = {x : y})

#### 03.02.02 Strip Abnormal Spaces

In [13]:
for x in df.columns:
  if df[x].dtypes == 'object':
    try:
      df[x] = df[x].str.strip()
    except:
      pass

#### 03.02.03 Convert Object Content to Proper Text

In [14]:
for x in df.columns:
  if df[x].dtypes == 'object':
    for a in df[x].unique():
      b = a.title()
      df[x] = df[x].replace(a, b)
  else:
    pass #'''

### 03.03 Coverting Data

#### 03.03.01 Convert to date

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     234005 non-null  int64  
 1   Date           234005 non-null  object 
 2   Time           234005 non-null  object 
 3   Ticket_Number  234005 non-null  float64
 4   Article        234005 non-null  object 
 5   Quantity       234005 non-null  float64
 6   Unit_Price     234005 non-null  float64
 7   Sales          234005 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 14.3+ MB


In [16]:
df['Date'] = pd.to_datetime(df['Date'])

#### 03.03.02 Convert to integer

#### 03.03.03 Convert to Object (if necessary)

#### 03.03.04 Replace 0 to Nan (if necessary)

In [17]:
#df = df.replace(0, np.nan)

## 04 Data Cleaning

### 04.01 Replacing Variable Component

### 04.02 Add New Variable

### 04.03 Inaccuracies

### 04.04 Handling Outliers
##### Data Distribution Check (Before)

In [18]:
'''fig = px.histogram(
    df,
    x = 'Quantity',
    y = 'Sales',
    marginal = 'box',
    hover_data = df.columns
)

fig.show() #'''

"fig = px.histogram(\n    df,\n    x = 'Quantity',\n    y = 'Sales',\n    marginal = 'box',\n    hover_data = df.columns\n)\n\nfig.show() #"

#### 04.04.01 Using IQR (Inter Quantile Range)

In [19]:
def iqr_thres(dataframe, column, th1 = 0.25, th3 = 0.75):
    quartile1 = dataframe[column].quantile(th1)
    quartile3 = dataframe[column].quantile(th3)
    iqr = quartile3 - quartile1
    upper_limit = quartile3 + 1.5 * iqr
    lower_limit = quartile1 - 1.5 * iqr

    return lower_limit, upper_limit

In [20]:
def check_outliers_iqr(dataframe, column):
    lower_limit, upper_limit = iqr_thres(dataframe, column)
    if dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)].any(axis = None):
        return True
    else:
        return False

In [21]:
def replace_iqr(dataframe, columns, th1 = 0.25, th3 = 0.75, replace = False):
  data = []

  for column in columns:
    if df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
      if column != 'Outcome':
        outliers_ = check_outliers_iqr(df, column)
        count = None
        lower_limit, upper_limit = iqr_thres(dataframe, column, th1, th3)

        if outliers_:
          count = dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)][column].count()
          if replace:
            if lower_limit < 0:
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
            else:
              dataframe.loc[(dataframe[column] < lower_limit), column] = lower_limit
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
        outliers_status = check_outliers_iqr(df, column)
        data.append([outliers_, outliers_status, count, column, lower_limit, upper_limit ])

  table = tabulate(data, headers = ['Outliers (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt = 'rst', numalign = 'right')
  print('Removing Outliers using IQR')
  print(table)

In [22]:
'''replace_iqr(
    dataframe = df,
    columns = [],
    replace = True
) #'''

'replace_iqr(\n    dataframe = df,\n    columns = [],\n    replace = True\n) #'

#### 04.04.02 Using Standard Deviation

In [23]:
def std_thres(dataframe, column):
    upper_limit = dataframe[column].mean() + 3 * dataframe[column].std()
    lower_limit = dataframe[column].mean() - 3 * dataframe[column].std()

    return lower_limit, upper_limit

In [24]:
def check_outliers_std(dataframe, column):
    lower_limit, upper_limit = iqr_thres(dataframe, column)
    if dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)].any(axis = None):
        return True
    else:
        return False

In [25]:
def replace_std(dataframe, columns, replace = False):
  data = []

  for column in columns:
    if df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
      if column != 'Outcome':
        outliers_ = check_outliers_std(dataframe, column)
        count = None
        lower_limit, upper_limit = std_thres(dataframe, column)

        if outliers_:
          count = dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)][column].count()
          if replace:
            if lower_limit < 0:
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
            else:
              dataframe.loc[(dataframe[column] < lower_limit), column] = lower_limit
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
        outliers_status = check_outliers_std(dataframe, column)
        data.append([outliers_, outliers_status, count, column, lower_limit, upper_limit])

  table = tabulate(data, headers = ['Outlier (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt = 'rst', numalign = 'right')
  print('Removing Outliers using 3 Standard Deviation')
  print(table)

In [26]:
'''replace_iqr(
    dataframe = df,
    columns = [],
    replace = False
) #'''

'replace_iqr(\n    dataframe = df,\n    columns = [],\n    replace = False\n) #'

### 04.05 Handling Missing / Zeros / Null
##### Filling missing value (numerical only) is better using median than mean or mode

#### 04.05.01 Detecting Zero Value

In [27]:
zero_columns = []

for x in df.columns:
  if df[x].dtypes == 'int64' or df[x].dtypes == 'float64':
    if (df[x] == 0).sum() != 0:
      print(x, ':', str((df[x] == 0).sum()))
      zero_columns.append(x)

Unnamed: 0 : 1
Unit_Price : 32
Sales : 32


#### 04.05.02 Detecting Nan / Non Available Value

In [28]:
nan_columns = []

for x in df.columns:
  if df[x].dtypes == 'object':
    if (df[x] == np.nan).sum() or (df[x] == '-').sum() != 0:
      print(x, ':', str((df[x] == np.nan).sum()))
      nan_columns.append(x)

#### 04.05.03 Replacing Zero with Mean (for numerical value if median value == 0), if necessary

In [29]:
'''cols = [
] #'''

'cols = [\n] #'

In [30]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].mean()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].mean()) #'

#### 04.05.04 Replacing Zero with Median (for numerical value if median value != 0), if necessary

In [31]:
'''cols = [
] #'''

'cols = [\n] #'

In [32]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].median()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].median()) #'

#### 04.05.05 Replacing Zero with Mode (for categorical / object value), if necessary

In [33]:
'''cols = [
] #'''

'cols = [\n] #'

In [34]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].mode()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].mode()) #'

### 04.06 Handling Incomplete Data

### 04.07 Handling Data Biases

### 04.08 Handling Duplicates

In [35]:
#df = df.drop_duplicates()

## 05 Enriching Data
#### take other dataset, inside or outside from related dataset / business

## 06 Data Validation
#### Verifying consistency, quality, and security of data

## 07 Exploration Data Analysis (Univariate)

### 07.01 Generate Data Profile

In [36]:
'''data_profile = ProfileReport(
    df,
    title = 'French Bakery Sales',
    dataset = {
        #'description' : 'This profiling report was generated for Analytics Vidhya Blog',
        #'copyright_holder' : 'Analytics Vidhya',
        #'copyright_year' : '2021',
        #'url' :  'https://www.analyticsvidhya.com/blog/',
    },
    correlations = {
        'pearson' : {'calculate' : True},
        'spearman' : {'calculate' : True},
        'kendall' : {'calculate' : True},
        'phi_k' : {'calculate' : True},
    },
)

data_profile #'''

"data_profile = ProfileReport(\n    df,\n    title = 'French Bakery Sales',\n    dataset = {\n        #'description' : 'This profiling report was generated for Analytics Vidhya Blog',\n        #'copyright_holder' : 'Analytics Vidhya',\n        #'copyright_year' : '2021',\n        #'url' :  'https://www.analyticsvidhya.com/blog/',\n    },\n    correlations = {\n        'pearson' : {'calculate' : True},\n        'spearman' : {'calculate' : True},\n        'kendall' : {'calculate' : True},\n        'phi_k' : {'calculate' : True},\n    },\n)\n\ndata_profile #"

In [37]:
df.head(3)

,Unnamed: 0,Date,Time,Ticket_Number,Article,Quantity,Unit_Price,Sales
0,0,2021-01-02,08:38,150040.0,Baguette,1.0,0.9,0.9
1,1,2021-01-02,08:38,150040.0,Pain Au Chocolat,3.0,1.2,3.6
2,4,2021-01-02,09:14,150041.0,Pain Au Chocolat,2.0,1.2,2.4


In [38]:
df.nunique()

Unnamed: 0       234005
Date                600
Time                683
Ticket_Number    136451
Article             149
Quantity             40
Unit_Price          123
Sales               357
dtype: int64

## 08 Store Data to Google Drive

In [39]:
'''cache_dir = CACHE_PATH + '/_cache.csv'
print(cache_dir)
df.to_csv(cache_dir, index = False)

sleep(60) #'''

"cache_dir = CACHE_PATH + '/_cache.csv'\nprint(cache_dir)\ndf.to_csv(cache_dir, index = False)\n\nsleep(60) #"

## 09 Table, Chart Drafting & Testing

In [40]:
fig = px.colors.sequential.swatches_continuous()
#fig.show()

In [41]:
colors_1 = px.colors.sequential.Reds
colors_2 = px.colors.sequential.RdBu
explode = tuple([0.015] * 50)
latitude = 0
longitude = 0
chart_theme = 'plotly_dark'
streamlit_theme = 'streamlit'
margin = {'r' : 20, 't' : 40, 'l' : 20, 'b' : 10}
cmap = 'magma_r'
title_x = 0
title_font_size = 18

In [42]:
title = 'Total Sales'
#st.subheader(title)
fig = go.Figure(go.Indicator(
  mode = 'number+delta',
  domain = {'x': [0, 1], 'y': [0, 1]},
  value = df['Sales'].sum(),
  number = {'prefix' : '€'},
))

fig.update_layout(
  #paper_bgcolor = 'lightgray',
  height = 200,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)
fig.show()

In [43]:
title = 'Product Sold'
#st.subheader(title)
fig = go.Figure(go.Indicator(
  mode = 'number+delta',
  domain = {'x': [0, 1], 'y': [0, 1]},
  value = df['Quantity'].sum(),
  number = {'valueformat' : ','},
))

fig.update_layout(
  #paper_bgcolor = 'lightgray',
  height = 200,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)
fig.show()

In [44]:
chart_df_1 = df.groupby(by = ['Article'], as_index = False)[['Sales', 'Quantity']].sum()
chart_df_1 = chart_df_1.sort_values('Sales', ascending = False)
chart_df_1[:10]

,Article,Sales,Quantity
140,Traditional Baguette,144756.05,117463.0
57,Formule Sandwich,34710.50,5181.0
32,Croissant,33579.50,29654.0
85,Pain Au Chocolat,31100.45,25236.0
7,Banette,24555.15,22732.0
4,Baguette,20481.40,22053.0
123,Sandwich Complet,13550.00,2888.0
126,Special Bread,13419.70,5456.0
141,Traiteur,12254.30,1456.0
72,Grand Far Breton,10713.50,1486.0


In [45]:
chart_df_1 = df.groupby(by = ['Article'], as_index = False)[['Sales', 'Quantity']].sum()
chart_df_1 = chart_df_1.sort_values('Sales', ascending = False)
chart_df_1 = chart_df_1[:20]

title = 'Sales Value (EUR) by Top 20 Product'
fig = go.Figure()
fig.add_trace(go.Bar(
  x = chart_df_1['Article'],
  y = chart_df_1['Sales'],
  name = 'Sales',
  marker = {'color': colors_2[0]},
))
fig.add_trace(go.Scatter(
  x = chart_df_1['Article'],
  y = chart_df_1['Quantity'],
  mode = 'lines+markers',
  name = 'Quantity',
  marker = {'color': colors_2[3]},
))
fig.update_layout(
  hovermode = 'x',
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)
fig.show()

In [46]:
df['Month & Year'] = df['Date'].dt.to_period('W')
linechart = pd.DataFrame(df.groupby(df['Month & Year'])[['Sales']].sum()).reset_index()
linechart['Month & Year'] = linechart['Month & Year'].astype(str)

# create time series chart

title = 'Time Series Sales & Profit Data'
#st.subheader(title)
fig_1 = go.Figure()
fig_1.add_trace(go.Scatter(
  x = linechart['Month & Year'],
  y = linechart['Sales'],
  mode = 'lines+markers',
  name = 'Sales',
  marker = {'color': colors_2[0]},
))

fig_1.update_layout(
  hovermode = 'x',
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)
fig_1.show()

## 10 Create Dashboard
### * template : ['ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff', 'ygridoff', 'gridon', 'none']
### * change layout https://plotly.com/python-api-reference/generated/plotly.graph_objects.Layout.html
### * more chart customization : https://plotly.com/python-api-reference/index.html

In [47]:
%%writefile app.py

import streamlit as st
import os
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings('ignore')

# 00 CREATING TAB TITLE

st.set_page_config(
  page_title = 'French Bakery Sales',
  page_icon = ':bar_chart:',
  layout = 'wide',
)


# 01 CREATING DASHBOARD TITLE

st.title(':bar_chart: French Bakery Sales Dashboard')
st.markdown('<style>div,block-container{padding-top:0rem;}<style>', unsafe_allow_html = True)


# 02 IMPORTING DATASET

dataset_path = 'https://raw.githubusercontent.com/azzindani/00_Data_Source/main/French_Bakery_Sales.csv'
df = pd.read_csv(dataset_path)

df['unit_price'] = df['unit_price'].str.replace(' €', '')
df['unit_price'] = df['unit_price'].str.replace(',', '.')
df['unit_price'] = df['unit_price'].astype('float')

df['sales'] = df['Quantity'] * df['unit_price']

for x in df.columns:
  y = x.title()
  df = df.rename(columns = {x : y})

for x in df.columns:
  if df[x].dtypes == 'object':
    try:
      df[x] = df[x].str.strip()
    except:
      pass

for x in df.columns:
  if df[x].dtypes == 'object':
    for a in df[x].unique():
      b = a.title()
      df[x] = df[x].replace(a, b)
  else:
    pass #'''


# 03 SETUP TEMPLATE & THEME

colors_1 = px.colors.sequential.BuPu_r
colors_2 = px.colors.sequential.BuPu_r
explode = tuple([0.015] * 50)
latitude = 0
longitude = 0
chart_theme = 'Darkmint'
streamlit_theme = 'streamlit'
margin = {'r' : 20, 't' : 40, 'l' : 20, 'b' : 10}
cmap = 'BuPu_r'
title_x = 0
title_font_size = 18


# 04 CREATING DATE PICKER

col_1, col_2 = st.columns((2))
df['Date'] = pd.to_datetime(df['Date'])

# Getting min & max date
start_date = pd.to_datetime(df['Date']).min()
end_date = pd.to_datetime(df['Date']).max()

with col_1:
  date_1 = pd.to_datetime(st.date_input('Start Date', start_date))

with col_2:
  date_2 = pd.to_datetime(st.date_input('End Date', end_date))

df = df[(df['Date'] >= date_1) & (df['Date'] <= date_2)] #'''


# 05 CREATING SIDEBAR FILTER


# 06 CREATING DATASET FILTER LOGIC


# 07 CREATING DASHBOARD

# create highlighted indicator

col_11, col_12 = st.columns((1, 4))

with col_11:
  title = 'Total Sales'
  #st.subheader(title)
  fig = go.Figure(go.Indicator(
    mode = 'number+delta',
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df['Sales'].sum(),
    number = {'prefix' : '€'},
  ))

  fig.update_layout(
    #paper_bgcolor = 'lightgray',
    height = 200,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

chart_df_1 = df.groupby(by = ['Article'], as_index = False)[['Sales', 'Quantity']].sum()
chart_df_1 = chart_df_1.sort_values('Sales', ascending = False)
chart_df_1 = chart_df_1[:20]

with col_12:
  title = 'Sales Value (EUR) by Top 20 Product'
  fig = go.Figure()
  fig.add_trace(go.Bar(
    x = chart_df_1['Article'],
    y = chart_df_1['Sales'],
    name = 'Sales',
    marker = {'color': colors_2[0]},
  ))
  fig.add_trace(go.Scatter(
    x = chart_df_1['Article'],
    y = chart_df_1['Quantity'],
    mode = 'lines+markers',
    name = 'Quantity',
    marker = {'color': colors_2[3]},
  ))
  fig.update_layout(
    hovermode = 'x',
    height = 400,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  fig.show()

  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

with col_11:
  title = 'Product Sold'
  #st.subheader(title)
  fig = go.Figure(go.Indicator(
    mode = 'number+delta',
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df['Quantity'].sum(),
    number = {'valueformat' : ','},
  ))

  fig.update_layout(
    #paper_bgcolor = 'lightgray',
    height = 200,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

col_21, col_22 = st.columns((4, 1))

# create time series chart

df['Month & Year'] = df['Date'].dt.to_period('W')
linechart = pd.DataFrame(df.groupby(df['Month & Year'])[['Sales']].sum()).reset_index()
linechart['Month & Year'] = linechart['Month & Year'].astype(str)

with col_21:
  title = 'Weekly Sales Data'
  #st.subheader(title)
  fig_1 = go.Figure()
  fig_1.add_trace(go.Scatter(
    x = linechart['Month & Year'],
    y = linechart['Sales'],
    mode = 'lines+markers',
    name = 'Sales',
    marker = {'color': colors_2[3]},
  ))

  fig_1.update_layout(
    hovermode = 'x',
    height = 500,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig_1, use_container_width = True, theme = streamlit_theme)

# Download original sample dataset

with col_22:
  title = '<p style = "font-size : 18px; font-weight : bold;">Sample Data</p>'
  st.markdown(title, unsafe_allow_html = True)
  st.write(df.iloc[:500, 1:20:2].style.background_gradient(cmap = cmap))
  csv = df.to_csv(index = False).encode('utf-8')
  st.download_button('Download Data', data = csv, file_name = 'Data.csv', mime = 'text/csv')

Overwriting app.py


## 11 Run Dashboard

In [48]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com
!npx localtunnel --port 8501

35.199.18.211
npx: installed 22 in 3.113s
your url is: https://legal-taxis-lead.loca.lt
^C
